# SQuAD FAISS Demo - Python API

This notebook demonstrates the **complete RAGDiff v2.0 workflow** using the Python API, from data preparation to comparison.

## Overview

This example compares two FAISS-based RAG systems using different embedding models:

1. **faiss-small**: `paraphrase-MiniLM-L3-v2` (17MB, 3 layers, fast)
2. **faiss-large**: `all-MiniLM-L12-v2` (120MB, 12 layers, more accurate)

## What You'll Learn

**Part 1: Data Preparation**
- Download and prepare the SQuAD dataset
- Build FAISS indices with different embedding models
- Generate query sets

**Part 2: RAGDiff API Usage**
- Execute queries against providers programmatically
- Compare results using LLM evaluation
- Analyze and export results

# Part 1: Data Preparation

First, we'll set up the demo data. This only needs to be run once.

## Prerequisites

**IMPORTANT: This notebook must be run from the `examples/squad-demo/` directory.**

Before running this notebook, ensure you have a uv environment set up:

```bash
# Navigate to the squad-demo directory
cd examples/squad-demo/

# Create virtual environment (if not already created in project root)
cd ../.. && uv venv && cd examples/squad-demo/

# Activate the environment
source ../../.venv/bin/activate  # On macOS/Linux
# or
..\..\..venv\Scripts\activate  # On Windows

# Install RAGDiff in editable mode (from project root)
cd ../.. && uv pip install -e . && cd examples/squad-demo/

# Start Jupyter from the squad-demo directory
jupyter notebook
```

Once Jupyter is running, you can proceed with the cells below.

## Step 1: Install Dependencies

In [15]:
# Install required packages using uv
import subprocess

packages = [
    "datasets",  # HuggingFace datasets for SQuAD
    "faiss-cpu",  # FAISS for vector search
    "sentence-transformers",  # Embedding models
    "numpy",  # Numerical operations
]

print("Installing required packages with uv...")

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call(["uv", "pip", "install", "-q", package])
        print(f"✓ {package} installed")

print("\nAll dependencies installed!")

Installing required packages with uv...
✓ datasets already installed
Installing faiss-cpu...
✓ faiss-cpu installed
✓ sentence-transformers already installed
✓ numpy already installed

All dependencies installed!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Step 2: Download and Prepare SQuAD Dataset

In [16]:
import json
from pathlib import Path

from datasets import load_dataset

# Set up paths - notebook should be run from examples/squad-demo/
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

output_file = data_dir / "documents.jsonl"
raw_file = data_dir / "squad_raw.json"

# Check if already prepared
if output_file.exists() and raw_file.exists():
    print(f"✓ Data already prepared at {data_dir}/")
    with open(output_file) as f:
        num_docs = sum(1 for _ in f)
    print(f"  - {num_docs} documents")
else:
    print("Loading SQuAD v2.0 dataset from HuggingFace...")
    dataset = load_dataset("squad_v2", split="validation")
    print(f"Loaded {len(dataset)} examples")

    # Extract unique contexts
    print("Extracting unique context paragraphs...")
    contexts_seen = set()
    documents = []

    for idx, example in enumerate(dataset):
        context = example["context"]
        if context in contexts_seen:
            continue
        contexts_seen.add(context)

        documents.append(
            {
                "id": f"squad_{len(documents)}",
                "text": context,
                "source": "SQuAD v2.0",
                "metadata": {
                    "title": example.get("title", "Unknown"),
                    "original_index": idx,
                },
            }
        )

    print(f"Found {len(documents)} unique context paragraphs")

    # Write documents
    print(f"Writing documents to {output_file}...")
    with open(output_file, "w", encoding="utf-8") as f:
        for doc in documents:
            f.write(json.dumps(doc, ensure_ascii=False) + "\n")

    # Save raw dataset for query generation
    print(f"Saving raw dataset to {raw_file}...")
    raw_data = {
        "examples": [
            {
                "id": ex["id"],
                "question": ex["question"],
                "context": ex["context"],
                "answers": ex["answers"],
                "title": ex.get("title", "Unknown"),
            }
            for ex in dataset
        ]
    }

    with open(raw_file, "w", encoding="utf-8") as f:
        json.dump(raw_data, f, indent=2, ensure_ascii=False)

    print("\n✓ Dataset preparation complete!")
    print(f"  Documents: {len(documents)}")
    print(f"  Q&A pairs: {len(dataset)}")

✓ Data already prepared at data/
  - 1204 documents


## Step 3: Build FAISS Indices

Build two FAISS indices with different embedding models:

In [17]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Use paths relative to examples/squad-demo/
data_dir = Path("data")
documents_file = data_dir / "documents.jsonl"

# Load documents
print("Loading documents...")
documents = []
with open(documents_file, encoding="utf-8") as f:
    for line in f:
        documents.append(json.loads(line.strip()))
print(f"Loaded {len(documents)} documents")

texts = [doc["text"] for doc in documents]

# Build small model index (fast but less accurate)
small_index_file = data_dir / "faiss_small.index"
if small_index_file.exists():
    print(f"\n✓ Small model index already exists at {small_index_file}")
else:
    print("\n" + "=" * 60)
    print("Building FAISS index with SMALL model (paraphrase-MiniLM-L3-v2)")
    print("=" * 60)

    print("Loading embedding model (small/fast)...")
    model = SentenceTransformer("paraphrase-MiniLM-L3-v2")

    print("Generating embeddings...")
    embeddings = model.encode(
        texts, show_progress_bar=True, batch_size=32, convert_to_numpy=True
    )
    embeddings = np.array(embeddings, dtype="float32")

    print(
        f"Generated {len(embeddings)} embeddings with dimension {embeddings.shape[1]}"
    )

    print("Building FAISS index with L2 distance...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print(f"Saving index to {small_index_file}...")
    faiss.write_index(index, str(small_index_file))

    print(f"✓ Small model index created ({index.ntotal} vectors, {index.d} dims)")
    print("  - 17MB model, 3 layers, fast but less accurate")

# Build large model index (slower but more accurate)
large_index_file = data_dir / "faiss_large.index"
if large_index_file.exists():
    print(f"\n✓ Large model index already exists at {large_index_file}")
else:
    print("\n" + "=" * 60)
    print("Building FAISS index with LARGE model (all-MiniLM-L12-v2)")
    print("=" * 60)

    print("Loading embedding model (larger/better quality)...")
    model = SentenceTransformer("all-MiniLM-L12-v2")

    print("Generating embeddings...")
    embeddings = model.encode(
        texts, show_progress_bar=True, batch_size=8, convert_to_numpy=True
    )
    embeddings = np.array(embeddings, dtype="float32")

    print(
        f"Generated {len(embeddings)} embeddings with dimension {embeddings.shape[1]}"
    )

    print("Building FAISS index with L2 distance...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print(f"Saving index to {large_index_file}...")
    faiss.write_index(index, str(large_index_file))

    print(f"✓ Large model index created ({index.ntotal} vectors, {index.d} dims)")
    print("  - 120MB model, 12 layers, slower but more accurate")

print("\n✓ All FAISS indices ready!")

Loading documents...
Loaded 1204 documents

✓ Small model index already exists at data/faiss_small.index

✓ Large model index already exists at data/faiss_large.index

✓ All FAISS indices ready!


## Step 4: Generate Query Sets

In [18]:
import random

# Use paths relative to examples/squad-demo/
data_dir = Path("data")
raw_file = data_dir / "squad_raw.json"

query_sets_dir = Path("domains/squad/query-sets")
query_sets_dir.mkdir(parents=True, exist_ok=True)

test_queries_file = query_sets_dir / "test-queries.txt"

if test_queries_file.exists():
    with open(test_queries_file) as f:
        num_queries = sum(1 for _ in f)
    print(f"✓ Query set already exists at {test_queries_file}")
    print(f"  - {num_queries} queries")
else:
    print("Generating query sets...")

    # Load raw dataset
    with open(raw_file, encoding="utf-8") as f:
        data = json.load(f)

    examples = data["examples"]

    # Filter answerable questions
    answerable = [ex for ex in examples if ex["answers"]["text"]]
    print(f"Found {len(answerable)} answerable questions")

    # Sample 100 questions
    random.seed(42)
    sampled = random.sample(answerable, min(100, len(answerable)))

    # Write test queries
    with open(test_queries_file, "w", encoding="utf-8") as f:
        for ex in sampled:
            f.write(ex["question"] + "\n")

    print(f"✓ Created {test_queries_file} ({len(sampled)} queries)")
    print("\nExample questions:")
    for i, ex in enumerate(sampled[:3], 1):
        print(f"  {i}. {ex['question']}")

print("\n✓ Data preparation complete! Ready to use RAGDiff API.")

✓ Query set already exists at domains/squad/query-sets/test-queries.txt
  - 100 queries

✓ Data preparation complete! Ready to use RAGDiff API.


# Part 2: RAGDiff API Usage

Now let's use the RAGDiff Python API to compare our providers.

## Setup: Import RAGDiff

In [19]:
# Diagnostic cell - run this first to see where you are
import os
from pathlib import Path

print(f"Current working directory: {os.getcwd()}")
print("\nContents of current directory:")
for item in sorted(os.listdir(".")[:10]):  # Show first 10 items
    print(f"  - {item}")

# Check if domains/squad exists relative to current directory
if Path("domains/squad").exists():
    print("\n✓ domains/squad/ found relative to current directory")
else:
    print("\n✗ domains/squad/ NOT found relative to current directory")
    print("  Will fix this in the next cell...")

Current working directory: /Users/mwk/Development/ansari-project/ragdiff/examples/squad-demo

Contents of current directory:
  - README.md
  - comparison-report-final.md
  - comparison-report-new.md
  - comparison-report.md
  - domains
  - pyproject.toml
  - ragdiff_squad_demo.egg-info
  - setup_notebook.sh
  - squad_demo_api.ipynb
  - uv.lock

✓ domains/squad/ found relative to current directory


In [20]:
import os
from datetime import datetime
from pathlib import Path

from rich.console import Console
from rich.table import Table

from ragdiff.comparison import compare_runs
from ragdiff.core.loaders import load_domain, load_provider, load_query_set
from ragdiff.execution import execute_run

console = Console()

# Fix the working directory - just set it to the correct absolute path
correct_dir = Path("/Users/mwk/Development/ansari-project/ragdiff/examples/squad-demo")
os.chdir(correct_dir)
print(f"Working directory: {Path.cwd()}")

# Configuration - paths relative to examples/squad-demo/
domain = "squad"
domains_dir = Path("domains")
providers = ["faiss-small", "faiss-large"]
query_set_name = "test-queries"

print(f"Domain: {domain}")
print(f"Domains directory: {domains_dir.absolute()}")
print(f"Providers: {providers}")
print(f"Query set: {query_set_name}")

Working directory: /Users/mwk/Development/ansari-project/ragdiff/examples/squad-demo
Domain: squad
Domains directory: /Users/mwk/Development/ansari-project/ragdiff/examples/squad-demo/domains
Providers: ['faiss-small', 'faiss-large']
Query set: test-queries


## 1. Explore Configurations

In [21]:
# Load domain configuration
domain_config = load_domain(domain, domains_dir)

print("=== Domain Configuration ===")
print(f"Name: {domain_config.name}")
print(f"Description: {domain_config.description}")
print(f"\nEvaluator Model: {domain_config.evaluator.model}")

# Load provider configurations
for provider_name in providers:
    provider_config = load_provider(domain, provider_name, domains_dir)
    print(f"\n=== Provider: {provider_name} ===")
    print(f"Tool: {provider_config.tool}")
    print(f"Config: {provider_config.config}")

=== Domain Configuration ===
Name: squad
Description: Example RAG comparison using SQuAD dataset with FAISS providers

Evaluator Model: anthropic/claude-sonnet-4-5

=== Provider: faiss-small ===
Tool: faiss
Config: {'index_path': 'data/faiss_small.index', 'documents_path': 'data/documents.jsonl', 'embedding_service': 'sentence-transformers', 'embedding_model': 'paraphrase-MiniLM-L3-v2', 'dimensions': 384}

=== Provider: faiss-large ===
Tool: faiss
Config: {'index_path': 'data/faiss_large.index', 'documents_path': 'data/documents.jsonl', 'embedding_service': 'sentence-transformers', 'embedding_model': 'all-MiniLM-L12-v2', 'dimensions': 384}


## 2. Load Query Set

In [22]:
queries = load_query_set(domain, query_set_name, domains_dir)

print(f"Query Set: {query_set_name}")
print(f"Total queries: {len(queries.queries)}")
print("\nFirst 5 queries:")
for i, query in enumerate(queries.queries[:5], 1):
    print(f"{i}. {query.text}")

Query Set: test-queries
Total queries: 100

First 5 queries:
1. Where did France focus its efforts to rebuild its empire?
2. What protestant religions made Northern European counties safe for Huguenot immigration?
3. BQP and QMA are examples of complexity classes most commonly associated with what type of Turing machine?
4. What was Apple Talk
5. What happens when bathocyroe and ocyropsis clap their lobes together?


## 3. Execute Runs

Execute queries against both providers:

In [23]:
def progress_callback(current, total, successes, failures):
    """Progress indicator"""
    if current % 10 == 0 or current == total:
        print(
            f"Progress: {current}/{total} queries ({successes} ok, {failures} failed)"
        )


runs = {}

for provider_name in providers:
    print(f"\n{'='*60}")
    print(f"Executing run: {provider_name}")
    print(f"{'='*60}\n")

    run = execute_run(
        domain=domain,
        provider=provider_name,
        query_set=query_set_name,
        label=f"{provider_name}-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        concurrency=10,
        per_query_timeout=30.0,
        progress_callback=progress_callback,
        domains_dir=domains_dir,
    )

    runs[provider_name] = run

    print(f"\n✓ Run completed: {run.label}")
    print(f"  Status: {run.status.value}")
    print(f"  Successes: {run.metadata.get('successes', 0)}")
    print(f"  Duration: {run.metadata.get('duration_seconds', 0):.2f}s")


Executing run: faiss-small

2025-10-28 11:47:05 - ragdiff.execution.executor - INFO - Starting run: domain=squad, provider=faiss-small, query_set=test-queries, concurrency=10
2025-10-28 11:47:05 - ragdiff.execution.executor - INFO - Loaded query set with 100 queries
2025-10-28 11:47:10 - ragdiff.providers.faiss - INFO - Initialized sentence-transformers model: paraphrase-MiniLM-L3-v2
2025-10-28 11:47:10 - ragdiff.providers.faiss - INFO - Loaded FAISS index with 1204 vectors from data/faiss_small.index
2025-10-28 11:47:10 - ragdiff.providers.faiss - INFO - Loaded 1204 documents from data/documents.jsonl
2025-10-28 11:47:10 - ragdiff.providers.factory - INFO - Created provider 'faiss-small' using tool 'faiss'
2025-10-28 11:47:10 - ragdiff.execution.executor - INFO - Created provider instance: FAISSProvider()
2025-10-28 11:47:10 - ragdiff.execution.executor - INFO - Run 06366d8d-939d-4bda-ac15-5d09891bd731 status: RUNNING
2025-10-28 11:47:10 - ragdiff.execution.executor - INFO - Executin

## 4. Compare Runs

Use LLM evaluation to compare the results:

In [24]:
def comparison_progress(current, total, successes, failures):
    """Progress indicator for comparison"""
    if current % 10 == 0 or current == total:
        print(
            f"Progress: {current}/{total} evaluations ({successes} ok, {failures} failed)"
        )


print(f"\n{'='*60}")
print(f"Comparing runs: {providers[0]} vs {providers[1]}")
print(f"{'='*60}\n")

comparison = compare_runs(
    domain=domain,
    run_ids=[run.id for run in runs.values()],
    concurrency=10,
    progress_callback=comparison_progress,
    domains_dir=domains_dir,
)

print("\n✓ Comparison completed")
print(f"  Duration: {comparison.metadata.get('duration_seconds', 0):.2f}s")


Comparing runs: faiss-small vs faiss-large

2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Starting comparison: domain=squad, run_ids=[UUID('06366d8d-939d-4bda-ac15-5d09891bd731'), UUID('a8cb3d67-b261-46e7-8f6c-2b48e4c3ae14')], model=None
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Auto-generated label: comparison-20251028-004
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Loaded 2 runs
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Using evaluator: model=anthropic/claude-sonnet-4-5
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - WARNING - Unknown model prefix for 'anthropic/claude-sonnet-4-5', skipping API key validation
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Evaluating 100 queries across 2 runs (concurrency=10)
2025-10-28 11:47:13 - ragdiff.comparison.evaluator - INFO - Executing 100 evaluations with concurrency=10
Progress: 10/100 evaluations (6 ok, 0 failed)
Progress: 20/100 evaluations (20 ok, 0 fail

## 5. Analyze Results

In [25]:
# Create summary table
table = Table(title="Comparison Results")
table.add_column("Provider", style="cyan")
table.add_column("Wins", style="green")
table.add_column("Losses", style="red")
table.add_column("Ties", style="yellow")
table.add_column("Avg Score", style="blue")
table.add_column("Avg Latency", style="magenta")

# Calculate stats
stats = {
    provider: {"wins": 0, "losses": 0, "ties": 0, "scores": [], "latencies": []}
    for provider in providers
}

for eval_result in comparison.evaluations:
    evaluation = eval_result.evaluation
    winner = evaluation.get("winner", "unknown")

    # Winner can be provider name, 'a', 'b', 'tie', or 'unknown'
    if winner == "tie":
        for provider in providers:
            stats[provider]["ties"] += 1
    elif winner == "a":
        stats[providers[0]]["wins"] += 1
        stats[providers[1]]["losses"] += 1
    elif winner == "b":
        stats[providers[1]]["wins"] += 1
        stats[providers[0]]["losses"] += 1
    elif winner in stats:
        # Winner is actual provider name
        stats[winner]["wins"] += 1
        for provider in providers:
            if provider != winner:
                stats[provider]["losses"] += 1
    # else: skip unknown winners

    # Try to get scores - they might be score_a/score_b or score_{provider_name}
    for provider in providers:
        score = (
            evaluation.get(f"score_{provider}")
            or evaluation.get("score_a" if provider == providers[0] else "score_b")
            or 0
        )
        if score:
            stats[provider]["scores"].append(score)

# Get latencies from runs
for provider, run in runs.items():
    for result in run.results:
        if result.duration_ms:
            stats[provider]["latencies"].append(result.duration_ms)

# Add rows
for provider in providers:
    avg_score = (
        sum(stats[provider]["scores"]) / len(stats[provider]["scores"])
        if stats[provider]["scores"]
        else 0
    )
    avg_latency = (
        sum(stats[provider]["latencies"]) / len(stats[provider]["latencies"])
        if stats[provider]["latencies"]
        else 0
    )

    table.add_row(
        provider,
        str(stats[provider]["wins"]),
        str(stats[provider]["losses"]),
        str(stats[provider]["ties"]),
        f"{avg_score:.1f}",
        f"{avg_latency:.1f}ms",
    )

console.print(table)

                       Comparison Results                       
┏━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Provider    ┃ Wins ┃ Losses ┃ Ties ┃ Avg Score ┃ Avg Latency ┃
┡━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ faiss-small │ 9    │ 24     │ 45   │ 63.7      │ 51.9ms      │
│ faiss-large │ 24   │ 9      │ 45   │ 68.7      │ 75.8ms      │
└─────────────┴──────┴────────┴──────┴───────────┴─────────────┘

# Export to JSON
output_file = Path("comparison_results.json")
with open(output_file, "w") as f:
    json.dump(comparison.model_dump(mode="json"), f, indent=2, default=str)

print(f"✓ Comparison exported to: {output_file}")
print(f"  File size: {output_file.stat().st_size / 1024:.1f} KB")

# Export to Markdown - using the same format as CLI
md_file = Path("comparison_results.md")

# Build markdown content using the same format as the CLI
lines = [
    f"# Comparison: {comparison.label}",
    "",
    f"**Domain:** {comparison.domain}",
    f"**Comparison ID:** {str(comparison.id)[:8]}...",
    f"**Model:** {comparison.evaluator_config.model}",
    f"**Temperature:** {comparison.evaluator_config.temperature}",
    "",
    "## Summary",
    "",
    f"- Total Evaluations: {comparison.metadata.get('total_evaluations', 0)}",
    f"- Successful: {comparison.metadata.get('successful_evaluations', 0)}",
    f"- Failed: {comparison.metadata.get('failed_evaluations', 0)}",
    "",
]

# Add provider statistics summary
lines.extend([
    "## Provider Statistics",
    "",
    "| Provider | Wins | Losses | Ties | Avg Score | Avg Latency |",
    "|----------|------|--------|------|-----------|-------------|",
])

for provider in providers:
    avg_score = (
        sum(stats[provider]["scores"]) / len(stats[provider]["scores"])
        if stats[provider]["scores"]
        else 0
    )
    avg_latency = (
        sum(stats[provider]["latencies"]) / len(stats[provider]["latencies"])
        if stats[provider]["latencies"]
        else 0
    )
    
    lines.append(
        f"| {provider} | {stats[provider]['wins']} | {stats[provider]['losses']} | "
        f"{stats[provider]['ties']} | {avg_score:.1f} | {avg_latency:.1f}ms |"
    )

lines.extend(["", "## Evaluations", ""])

# Add individual evaluations
for i, eval_result in enumerate(comparison.evaluations[:20], 1):  # Show first 20
    lines.append(f"### {i}. {eval_result.query}")
    lines.append("")
    
    if eval_result.reference:
        lines.append(f"**Reference:** {eval_result.reference}")
        lines.append("")
    
    if "winner" in eval_result.evaluation:
        lines.append(
            f"**Winner:** {eval_result.evaluation.get('winner', 'unknown')}"
        )
        lines.append("")
    
    if "reasoning" in eval_result.evaluation:
        reasoning = eval_result.evaluation.get('reasoning', '')
        lines.append(f"**Reasoning:** {reasoning}")
        lines.append("")
    
    # Add scores if present
    for provider in providers:
        score = (
            eval_result.evaluation.get(f"score_{provider}")
            or eval_result.evaluation.get("score_a" if provider == providers[0] else "score_b")
        )
        if score:
            lines.append(f"**Score {provider}:** {score}")
    
    if "_metadata" in eval_result.evaluation:
        metadata = eval_result.evaluation["_metadata"]
        lines.append(
            f"**Cost:** ${metadata.get('cost', 0):.4f}, "
            f"**Tokens:** {metadata.get('total_tokens', 0)}"
        )
    lines.append("")

if len(comparison.evaluations) > 20:
    lines.append(f"*... and {len(comparison.evaluations) - 20} more evaluations*")

# Write markdown file
markdown = "\n".join(lines)
with open(md_file, "w") as f:
    f.write(markdown)

print(f"✓ Comparison exported to: {md_file}")
print(f"  File size: {md_file.stat().st_size / 1024:.1f} KB")

In [ ]:
# Export to JSON
output_file = Path("comparison_results.json")
with open(output_file, "w") as f:
    json.dump(comparison.model_dump(mode="json"), f, indent=2, default=str)

print(f"✓ Comparison exported to: {output_file}")
print(f"  File size: {output_file.stat().st_size / 1024:.1f} KB")

# Export to Markdown using the shared utility
from ragdiff.display.formatting_v2 import (
    calculate_provider_stats_from_runs,
    save_comparison_markdown,
)

md_file = Path("comparison_results.md")

# Calculate provider stats from runs and comparison
provider_stats = calculate_provider_stats_from_runs(runs, comparison)

# Save markdown using the utility function
save_comparison_markdown(
    comparison,
    md_file,
    provider_stats=provider_stats,
    max_evaluations=20,  # Show first 20 evaluations in notebook
)

print(f"✓ Comparison exported to: {md_file}")
print(f"  File size: {md_file.stat().st_size / 1024:.1f} KB")

## Summary

This notebook demonstrated the complete RAGDiff v2.0 workflow:

**Part 1: Data Preparation**
- ✓ Downloaded SQuAD dataset from HuggingFace
- ✓ Built FAISS indices with different embedding models
- ✓ Generated query sets for testing

**Part 2: RAGDiff API**
- ✓ Executed queries against multiple providers
- ✓ Compared results using LLM evaluation
- ✓ Analyzed and exported results

### Key Takeaways

- The **large model** (all-MiniLM-L12-v2) typically wins more comparisons but is slower
- The **small model** (paraphrase-MiniLM-L3-v2) is much faster but less accurate
- This demonstrates the classic **quality vs speed tradeoff** in embedding models

### Next Steps

- Create custom query sets for your domain
- Try different embedding models (e.g., all-mpnet-base-v2)
- Adjust concurrency for faster execution
- Experiment with different LLM evaluators

For more information, see the [RAGDiff documentation](../../CLAUDE.md).